In [ ]:
!pip3 install ultralytics

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO


In [2]:
model = YOLO('yolov8n.pt')  # You can use 'yolov8n.pt', 'yolov8s.pt', etc.


In [3]:
vehicle_classes = [1, 2, 3, 5, 7]  # 1: Bicycle, 2: Car, 3: Motorcycle, 5: Bus, 7: Truck


In [4]:
def detect_vehicles(frame):
    results = model(frame)  # Perform inference using YOLOv8
    vehicle_boxes = []
    
    # Process detection results
    for result in results:
        for bbox, cls_id, conf in zip(result.boxes.xyxy, result.boxes.cls, result.boxes.conf):
            if int(cls_id) in vehicle_classes and conf > 0.5:  # Filter by class and confidence
                x1, y1, x2, y2 = map(int, bbox)  # Get bounding box coordinates
                vehicle_boxes.append([x1, y1, x2, y2])
    
    return vehicle_boxes

In [5]:
def generate_heatmap(frame, heatmap):
    vehicle_boxes = detect_vehicles(frame)
    
    # Increment intensity in the heatmap for detected vehicle areas
    for (x1, y1, x2, y2) in vehicle_boxes:
        heatmap[y1:y2, x1:x2] += 1
    
    # Normalize the heatmap for visualization
    normalized_heatmap = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX)
    colored_heatmap = cv2.applyColorMap(np.uint8(normalized_heatmap), cv2.COLORMAP_JET)
    
    return colored_heatmap

In [6]:
def process_live_feed(PATHH):
    cap = cv2.VideoCapture(PATHH)  # Use 0 for default camera, or replace with video file path
    ret, first_frame = cap.read()
    
    # Initialize heatmap as an empty array with the same size as the video frame
    heatmap = np.zeros((first_frame.shape[0], first_frame.shape[1]), dtype=np.float32)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Generate the heatmap based on live vehicle detection
        live_heatmap = generate_heatmap(frame, heatmap)
        
        # Overlay the heatmap on the current video frame
        result_frame = cv2.addWeighted(frame, 0.6, live_heatmap, 0.4, 0)
        
        # Display the resulting frame with the heatmap overlay
        cv2.imshow("Live Traffic Congestion Heatmap", result_frame)
        
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [7]:
process_live_feed('video.mp4')


0: 384x640 19 cars, 4 buss, 1 truck, 38.3ms
Speed: 2.9ms preprocess, 38.3ms inference, 4.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 4 buss, 2 trucks, 32.3ms
Speed: 1.4ms preprocess, 32.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22 cars, 4 buss, 1 truck, 30.5ms
Speed: 1.4ms preprocess, 30.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21 cars, 4 buss, 1 truck, 28.7ms
Speed: 1.2ms preprocess, 28.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 20 cars, 4 buss, 30.5ms
Speed: 1.4ms preprocess, 30.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21 cars, 4 buss, 30.5ms
Speed: 1.5ms preprocess, 30.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 20 cars, 4 buss, 30.5ms
Speed: 1.4ms preprocess, 30.5ms inference, 0.6ms postprocess per image at shape

: 